etri api를 사용하여 형태소 분석 후 명사만 가져와 토큰화한다.  
* 형태소 분석 (문어/구어) : "morp"

# import library

In [2]:
import urllib3
import json
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 100)


import warnings
warnings.filterwarnings("ignore")

# upload article data
article의 title과 content를 합친 내용으로 토큰화 한다.

In [3]:
df1 = pd.read_json('/fastcampus-data/articles/articels_only_contents.json')
df = df1.copy()

# article의 title과 content를 합쳐 full_content라는 새로운 컬럼에 담는다.
df["full_content"] = df['title'] + ' ' + df['content_tag_removed']
df = df[['id', 'title', 'full_content']].reset_index(drop=True)

In [4]:
# api_key = '8e216d4f-bfae-4cc4-ae6f-25f2dc4968c5' # 혜빈 키
api_key = 'de384842-9f74-42c9-a0e5-036ecd76eab5' # 영수님 키

언어 분석 기술 문어/구어 중 한가지만 선택해 사용

In [5]:
# // 언어 분석 기술(문어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 

# preprocessing

### 1. api로 불러온 data에서 필요한 정보 추출

In [6]:
class ETRISentenceAnalysis:
    def __init__(self, api_key: dict, url: str):
        self.api_key = api_key
        self.url = url

        self.http = urllib3.PoolManager()
  
    def _make_request_json(self, text: str, analysis_code: int) -> dict:
        return {
          "access_key": self.api_key,
          "argument": {
              "text": text,
              "analysis_code": analysis_code
              }
          }

    def _request(self, request_json: dict) -> object:
        return self.http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8"},
            body=json.dumps(request_json)
        )

    # 형태소 분석 결과 출력
    def get_analyzed_sentence(self, sentence: str, analysis_code: int) -> dict:
        def _get_return_object(sentence: str) -> dict:
            data = str(response.data, 'utf-8')
            data = json.loads(data)


            return data.get('return_object') 

        request_json = self._make_request_json(sentence, analysis_code)
        response = self._request(request_json)

        assert response.status == 200, f'Error {response.status}'

        return _get_return_object(response)

In [7]:
etri_sentence_analysis = ETRISentenceAnalysis(api_key, openApiURL)

### 2. 한글을 제외한 문자 제거

In [8]:
import re    
df['full_content'] = df['full_content'].apply(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))

### 3. 글자가 20글자 이하이거나, 내용이 NoneType인 아티클 제거
총 31개. id로 접근함

In [9]:
# contents 글자가 20글자 이하인 아티클
id_less_20 = []
for i in range(len(df['full_content'])):
    if len(df['full_content'][i]) < 20:
#         print(i, df['full_content'][i])
        id_less_20.append(df['id'][i])

        
print('len(id_less_20) = ',len(id_less_20))    
print('id_less_20 = ',id_less_20)    

# none type인 아티클
type_none_idx = [1723, 1724, 1804, 1826, 2245, 2318, 2319, 2377, 2431, 2442, 2455, 2456, 2542, 2561, 2563, 2567, 2655, 2701, 2768, 2832, 2857, 2937, 2938, 2954, 3065, 3088] 
new_id = []
# for i in range(len(df['content_tag_removed'])):
for j in type_none_idx:
    new_id.append(df.iloc[j].id)
    
print('len(new_id) = ',len(new_id))    
print('new_id = ', new_id)    
print()

drop_id_list = list(set(id_less_20 + new_id))
print('len(drop_id_list) = ',len(drop_id_list))
print('drop_id_list = ', drop_id_list)

len(id_less_20) =  0
id_less_20 =  []
len(new_id) =  26
new_id =  [15107, 15376, 18326, 17194, 22131, 27831, 27888, 24527, 29398, 29528, 28451, 28471, 30618, 29157, 27366, 27451, 29891, 33399, 32309, 33743, 33446, 34361, 34422, 35145, 35259, 36137]

len(drop_id_list) =  26
drop_id_list =  [15107, 15376, 18326, 30618, 28451, 33446, 36137, 17194, 32309, 28471, 27831, 34361, 27451, 35259, 29891, 35145, 24527, 33743, 29398, 29528, 29157, 27366, 27888, 22131, 34422, 33399]


# corpus에 형태소 분석 결과 담기

In [10]:
corpus = []

for i in tqdm(range(len(df))):
    con = df['full_content'].iloc[i]
    # 형태소 분석
    data = etri_sentence_analysis.get_analyzed_sentence(con, 'morp')
    corpus.append(data)
       
len(corpus)

100%|██████████| 3438/3438 [18:59<00:00,  3.02it/s]


3438

# dataframe

In [11]:
# dataframe 초기화
new_df = pd.DataFrame(df[['id','title','full_content']])
new_df['tokens'] = None

# dataframe 각 토큰 담기
for i in tqdm(range(len(corpus))):
    nouns = []
    res3 = corpus[i]['sentence'][0]['morp'] # 형태소 결과
    
    for j in res3:
        # 형태소가 명사이고 2글자 이상인 것만 담기
        if j['type'] == 'NNG' and len(j['lemma']) > 1: 
            nouns.append(j['lemma'])
            new_df['tokens'].iloc[i] = nouns

100%|██████████| 3438/3438 [01:56<00:00, 29.56it/s]


In [12]:
# 중복되는 토큰제거
new_df['tokens'] = new_df['tokens'].apply(lambda x: list(set(x)))

In [13]:
new_df.head()

,id,title,full_content,tokens
0,43,디스크탈출증 재흡수 케이스,디스크탈출증 재흡수 케이스 케이스의 종류 치료성공 케이스 환자 성별 나이...,"[요추, 보존, 근력, 여자, 약화, 다리, 우측, 병력, 결론, 흡수, 운동, 계..."
1,61,여기에 질문하는거 맞나요?,여기에 질문하는거 맞나요 수고하십니다 자보환자가 한약 치료 끝나고 보험제제를 계...,"[제제, 한약, 자보, 병원, 치료, 환자, 제품, 수고, 인정, 엑스, 보험, 질문]"
2,66,뒷목통증 좌측방사통을 주소로 내원한 환자,뒷목통증 좌측방사통을 주소로 내원한 환자 케이스의 종류 치료 성공 케이스 ...,"[진료, 모근, 남자, 소실, 신전, 우측, 특별, 병력, 상기, 운동, 수술, 저..."
3,46,퇴행성 관절염 환자 무릎에 물이 찬 경우.,퇴행성 관절염 환자 무릎에 물이 찬 경우 한의원에서 대응해서 할 수 있는 치료주사...,"[퇴행, 경우, 대응, 무릎, 치료, 환자, 주사, 관절, 의원]"
4,51,비전형적인 Cauda equina syn.,비전형적인 케이스의 종류 함께 논의하고 싶은 케이스 환자 ...,"[여자, 우측, 다리, 병력, 결론, 운동, 회음, 본인, 종류, 소변, 동반, 스..."


# save to json

In [29]:
# new_df.to_json('/home/user_4/CBF/Token/final_tokens_article.json')